In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/advertising-cmaes/advertising.csv


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import csv 
import matplotlib.pyplot as plt
import pickle
import random
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
import h5py
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
!pip install xcs

     |████████████████████████████████| 53 kB 1.2 MB/s eta 0:00:011


In [4]:
import xcs
from xcs import XCSAlgorithm
from xcs.scenarios import MUXProblem, ScenarioObserver
import logging
logging.root.setLevel(logging.INFO)
import random
from xcs.scenarios import Scenario
from xcs.bitstrings import BitString

In [5]:
df = pd.read_csv("../input/advertising-cmaes/advertising.csv")
df.head(5)

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,2016-03-27 00:53:11,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,2016-04-04 01:39:02,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,2016-03-13 20:35:42,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,2016-01-10 02:31:19,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,2016-06-03 03:36:18,0


In [6]:
df.drop(['City', 'Country','Timestamp','Ad Topic Line'], axis = 1, inplace=True) 

In [7]:
print(df["Daily Internet Usage"].min())

104.78


In [8]:
df["Daily Time Spent on Site"] = pd.cut(df["Daily Time Spent on Site"],4,labels=[0,1,2,3])

In [9]:
df["Age"] = pd.cut(df["Age"],3,labels=[0,1,2])

In [10]:
df["Area Income"] = pd.cut(df["Area Income"],4,labels=[0,1,2,3])

In [11]:
df["Daily Internet Usage"] = pd.cut(df["Daily Internet Usage"],3,labels=[0,1,2])

In [12]:
print(df.head(10))

  Daily Time Spent on Site Age Area Income Daily Internet Usage  Male  \
0                        2   1           2                    2     0   
1                        3   0           3                    1     1   
2                        2   0           2                    2     0   
3                        2   0           2                    2     1   
4                        2   1           3                    2     0   
5                        1   0           2                    2     1   
6                        3   0           2                    1     0   
7                        2   2           0                    0     1   
8                        2   0           3                    2     1   
9                        2   0           2                    1     1   

   Clicked on Ad  
0              0  
1              0  
2              0  
3              0  
4              0  
5              0  
6              0  
7              1  
8              0  
9     

In [13]:
df1 = pd.get_dummies(df["Daily Time Spent on Site"], prefix = 'Daily Time Spent on Site')
df2 = pd.get_dummies(df["Age"], prefix = 'Age')
df3 = pd.get_dummies(df["Area Income"], prefix = 'Area Income')
df4 = pd.get_dummies(df["Daily Internet Usage"], prefix = 'Daily Internet Usage')

In [14]:
DataFrame = pd.concat([df["Clicked on Ad"],df["Male"],df1, df2,df3,df4], axis=1)
print(DataFrame.head(10))

   Clicked on Ad  Male  Daily Time Spent on Site_0  \
0              0     0                           0   
1              0     1                           0   
2              0     0                           0   
3              0     1                           0   
4              0     0                           0   
5              0     1                           0   
6              0     0                           0   
7              1     1                           0   
8              0     1                           0   
9              0     1                           0   

   Daily Time Spent on Site_1  Daily Time Spent on Site_2  \
0                           0                           1   
1                           0                           0   
2                           0                           1   
3                           0                           1   
4                           0                           1   
5                           1          

In [15]:
pid_dataset = DataFrame.values
arr = []
for i in range(len(pid_dataset)):
    arr.append(''.join([str(x) for x in pid_dataset[i]]))

In [16]:
class Advertising(Scenario):
    def __init__(self, training_cycles=10000, input_size=500, inputs=[]):
        self.input_size = input_size
        self.possible_actions = (1, 0)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle_index = random.randrange(input_size)
        self.needle_value = None
        self.inputs = inputs
    
    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
    
    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        self.needle_index = random.randrange(self.input_size)
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        random.seed()
        i = random.randint(0,len(self.inputs) - 1)
        c_feat = BitString(self.inputs[i][:-1])
        self.needle_value = self.inputs[i][-1]
        return c_feat
        
    def execute(self, action):
        self.remaining_cycles -= 1
        return int(action) == int(self.needle_value)

In [17]:
inp = len(arr[0])-1
problem = Advertising(training_cycles=10000, input_size=inp, inputs = arr)

algorithm = xcs.XCSAlgorithm()

# Default parameter settings in test()
algorithm.exploration_probability = .1
algorithm.discount_factor = 0
algorithm.do_ga_subsumption = True
algorithm.do_action_set_subsumption = True

# Modified parameter settings
algorithm.ga_threshold = 1
algorithm.crossover_probability = .5
algorithm.wildcard_probability = .5
algorithm.deletion_threshold = 2 
algorithm.subsumption_threshold = 10         # theta_sub
algorithm.mutation_probability = .003

In [18]:
model = algorithm.new_model(ScenarioObserver(problem))
history = model.run(ScenarioObserver(problem), learn=True)

In [19]:
print(model)

11#1#0010#00##1 => 1
    Time Stamp: 5265
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
#0#0#1100010#01 => 1
    Time Stamp: 5823
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
110#1###11###0# => 1
    Time Stamp: 5984
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1#01#0010100#1# => 1
    Time Stamp: 6687
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1##1000#11####1 => 1
    Time Stamp: 8129
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
10##001####000# => 1
    Time Stamp: 8255
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
#101###0#0##000 => 0
 

In [21]:
for rule in model:
    if rule.fitness > .5 and rule.experience >= 10:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness) 

#############1# => 0  [0.99869]
#############00 => 1  [1.00000]
00000###000##00 => 0  [0.97043]
#1#00###0#0##00 => 0  [0.94647]
#########0###10 => 1  [0.99817]
0#00#0##0####00 => 0  [0.81027]
##############1 => 0  [0.99760]
###0####000###1 => 1  [0.72217]
